In [ ]:
from unsloth import FastLanguageModel #type: ignore
import torch #type: ignore
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True,
    loftq_config=None,
)


In [ ]:
from unsloth.chat_templates import get_chat_template #type: ignore
from datasets import Dataset #type: ignore
import json

def load_and_convert_to_llama_format(file_path):
    with open(file_path, 'r') as infile:
        data = json.load(infile)

    llama_data = []

    for conversation in data:
        conv = []
        for entry in conversation:
            if "from" not in entry or "value" not in entry:
                print(f"Missing keys in entry: {entry}")
                continue 

            conv.append({
                "role": entry["from"],
                "content": entry["value"]
            })
        if conv:
            llama_data.append({"conversation": conv})

    return llama_data
local_file_path = "/content/socratic_teaching_conversations.json"
llama_data = load_and_convert_to_llama_format(local_file_path)
if not llama_data:
    print("No valid conversations found in the input data.")
else:
    dataset = Dataset.from_list(llama_data)
    tokenizer = get_chat_template(
        tokenizer,
        chat_template="chatml",
        mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
        map_eos_token=True,
    )

    def formatting_prompts_func(examples):
        texts = []
        conversations = examples['conversation']

        for conversation in conversations:
            formatted_conversation = []
            for message in conversation:
                formatted_conversation.append({
                    "from": message['role'],
                    "value": message['content']
                })
            text = tokenizer.apply_chat_template(formatted_conversation, tokenize=False, add_generation_prompt=False)
            texts.append(text)

        return {"text": texts}
    dataset = dataset.map(formatting_prompts_func, batched=True)
    formatted_texts = dataset['text']
    print("Formatted texts:", formatted_texts)

    print("Dataset formatted successfully.")

In [ ]:
from trl import SFTTrainer #type: ignore
from transformers import TrainingArguments #type: ignore
from unsloth import is_bfloat16_supported #type: ignore

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
FastLanguageModel.for_inference(model)
user_question = input("Ask a question: ")
inputs = tokenizer(
    [user_question],
    return_tensors="pt"
).to("cuda")
input_length = inputs['input_ids'].shape[1]
outputs = model.generate(**inputs, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs[:, input_length:], skip_special_tokens=True)
print(decoded_outputs)

In [ ]:
FastLanguageModel.for_inference(model)
alpaca_prompt = """You are a helpful assistant. Given a specific instruction, provide an appropriate continuation based on the input.
Instruction: {}
Input: {}
Output: {}
"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a socratic teacher, always complete response within 100 words , How do i learn about the fibonnaci sequence.", # instruction
        "",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer #type: ignore
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer)

In [ ]:
model.save_pretrained("lora_model") 
tokenizer.save_pretrained("lora_model")

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)